# Multi-Agent Orchestration with MLflow Tracing and Agent-as-a-Judge Evaluation

## 📋 Tutorial Overview

This notebook demonstrates how to build, trace, and evaluate a multi-agent AI system using MLflow's advanced observability and evaluation capabilities. We'll create an orchestrator agent that intelligently delegates tasks to specialized worker agents, then use Agent-as-a-Judge to automatically evaluate the system's performance.

---

## 🎯 Learning Objectives

By the end of this tutorial, you will be able to:

1. **Build a Multi-Agent Architecture** with an orchestrator pattern
2. **Implement MLflow Tracing** to capture hierarchical agent execution flows
3. **Create Agent-as-a-Judge evaluators** using `make_judge` API
4. **Analyze execution patterns** including tool usage, performance, and output quality
5. **Visualize traces** in the MLflow UI for debugging and optimization

---

## 📦 Prerequisites

### Required Packages

```bash
uv sync
```

Or install manually:
```bash
pip install mlflow>=3.5.1
pip install autogen-agentchat>=0.7.5
pip install autogen-ext[openai]>=0.7.5
pip install python-dotenv>=1.2.1
pip install litellm>=1.80.7  
pip install rich>=14.2.0     # For beautiful console output
pip install databricks-sdk   # For Databricks integration
```

### Environment Setup

Create a `.env` file with your Databricks credentials:

```env
DATABRICKS_HOST=https://your-workspace.cloud.databricks.com
DATABRICKS_TOKEN=your-databricks-token
```

### MLflow Server

Start your local MLflow tracking server:

```bash
uv run mlflow server --host 127.0.0.1 --port 5000
```

Or with SQLite DB:
```bash
uv run mlflow ui --backend-store-uri sqlite:///mlflow.db --port 5000
```

---

## 🔌 MLflow Provider Support

**What this tutorial uses:** Databricks Foundation Model APIs (OpenAI-compatible)

**MLflow also supports:**
- ✅ **OpenAI** - Direct OpenAI API
- ✅ **Azure OpenAI** - Azure-hosted OpenAI models
- ✅ **Anthropic** - Claude models  
- ✅ **Google Gemini** - Vertex AI
- ✅ **AWS Bedrock** - Amazon's managed LLMs
- ✅ **Ollama** - Local open-source models
- ✅ **HuggingFace** - Hosted inference endpoints

**To use a different provider:**

```python
# Databricks (what we use here)
from autogen_ext.models.openai import OpenAIChatCompletionClient
model_client = OpenAIChatCompletionClient(
    model="databricks-gpt-5-2",
    base_url=f"{DATABRICKS_HOST}/serving-endpoints",
    api_key=DATABRICKS_TOKEN
)

# OpenAI
from autogen_ext.models.openai import OpenAIChatCompletionClient
model_client = OpenAIChatCompletionClient(model="gpt-4o", api_key="...")

# Azure OpenAI
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
model_client = AzureOpenAIChatCompletionClient(
    model="gpt-4o",
    azure_endpoint="https://your-resource.openai.azure.com/",
    api_key="..."
)

# Ollama (local)
from autogen_ext.models.openai import OpenAIChatCompletionClient
model_client = OpenAIChatCompletionClient(
    model="llama3.2",
    base_url="http://localhost:11434/v1",
    api_key="ollama"
)
```

**📖 Always check the latest documentation:**
- [MLflow GenAI Documentation](https://mlflow.org/docs/latest/llms/index.html)
- [AutoGen Model Clients](https://microsoft.github.io/autogen/dev/user-guide/core-user-guide/models.html)
- [Databricks Foundation Model APIs](https://docs.databricks.com/en/machine-learning/foundation-models/index.html)

⚠️ **Important:** MLflow and AutoGen are rapidly evolving. Always verify compatibility with the latest versions before starting your project.

---

# Section 1: Setup & Configuration

---

## 📌 STEP 1: Configure Environment Variables

**What this does:**
- Sets up Databricks credentials as environment variables
- Configures the Databricks host URL and personal access token
- Specifies the model to use (Databricks Foundation Model)
- Makes credentials available to all agents and the judge system

In [ ]:
import os
from dotenv import load_dotenv

# Load from .env file if available
load_dotenv()

# Databricks credentials - set these or use .env file
DATABRICKS_HOST = os.getenv("DATABRICKS_HOST", "https://your-workspace.cloud.databricks.com")
DATABRICKS_TOKEN = os.getenv("DATABRICKS_TOKEN", "your-databricks-token")

# Set environment variables for downstream use
os.environ["DATABRICKS_HOST"] = DATABRICKS_HOST
os.environ["DATABRICKS_TOKEN"] = DATABRICKS_TOKEN

# Model configuration - Databricks Foundation Model
DATABRICKS_MODEL = "databricks-gpt-5-2"

print("✅ Environment variables configured successfully")
print(f"🏢 Databricks Host: {DATABRICKS_HOST[:50]}...")
print(f"🤖 Model: {DATABRICKS_MODEL}")

---

## 📌 STEP 2: Import Required Dependencies

**What this does:**
- Imports core libraries for async operations, environment management, and MLflow
- Loads AutoGen v0.4 components for agent creation and tool management
- Imports MLflow's Agent-as-a-Judge functionality
- Configures logging to prevent verbose output


---

📖 **Latest docs:** [MLflow Changelog](https://github.com/mlflow/mlflow/releases) | [AutoGen Releases](https://github.com/microsoft/autogen/releases)

---

In [ ]:
import os
import asyncio
import mlflow
from dotenv import load_dotenv

# --- AutoGen v0.4 Imports ---
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.tools import AgentTool
from autogen_ext.models.openai import OpenAIChatCompletionClient  # OpenAI-compatible client for Databricks
from autogen_core.model_context import BufferedChatCompletionContext
from autogen_ext.ui import RichConsole

# --- MLflow Judge Imports ---
from mlflow.genai.judges import make_judge

print("✅ All dependencies imported successfully")
print(f"📦 MLflow version: {mlflow.__version__}")

---

## 📌 STEP 3: Configure MLflow Tracking

**What this does:**
- Loads environment variables from `.env` file (if available)
- Sets the MLflow tracking URI to connect to the local MLflow server
- Creates or connects to an experiment named "new-demo"
- Verifies the connection to the tracking server

**✅ Expected Output:**
```
✅ MLflow Tracking URI: http://127.0.0.1:5000
```

**💡 Access UI:**
Access the MLflow UI by navigating to `http://127.0.0.1:5000` in your browser to visualize traces in real-time!

In [8]:
load_dotenv()
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("MLFlow-demo")
print(f"✅ MLflow Tracking URI: {mlflow.get_tracking_uri()}")

2025/12/02 16:02:48 INFO mlflow.tracking.fluent: Experiment with name 'MLFlow-demo' does not exist. Creating a new experiment.


✅ MLflow Tracking URI: http://127.0.0.1:5000


---

## 📌 STEP 4: Initialize Databricks Model Client

**What this does:**
- Creates a Databricks Foundation Model client that all agents will share
- Uses the OpenAI-compatible API endpoint provided by Databricks
- Sets temperature to 0 for deterministic, reproducible responses

**💡 Available Databricks Models:**
- `databricks-gpt-5-2` - GPT-5.2 hosted on Databricks (used in this tutorial)
- `databricks-meta-llama-3-3-70b-instruct` - Llama 3.3 70B
- `databricks-meta-llama-3-1-405b-instruct` - Llama 3.1 405B
- `databricks-mixtral-8x7b-instruct` - Mixtral 8x7B
- `databricks-dbrx-instruct` - DBRX (Databricks' own model)

In [ ]:
# --- DATABRICKS CLIENT SETUP ---
# Databricks Foundation Model APIs are OpenAI-compatible
model_client = OpenAIChatCompletionClient(
    model=DATABRICKS_MODEL,
    base_url=f"{DATABRICKS_HOST}/serving-endpoints",
    api_key=DATABRICKS_TOKEN,
    temperature=0,
)

print("✅ Databricks model client initialized successfully")
print(f"🤖 Using model: {DATABRICKS_MODEL}")
print(f"🔗 Endpoint: {DATABRICKS_HOST}/serving-endpoints")

---

## 📌 STEP 5: Enable MLflow AutoGen Autologging

**What this does:**
- Activates MLflow's automatic logging for AutoGen agents
- Automatically captures all agent interactions, tool calls, and LLM requests
- Creates hierarchical traces showing the complete execution flow

**💡 Key Concept:**
`mlflow.autogen.autolog()` is a "magic" function that automatically instruments all AutoGen agents. Once enabled, MLflow will:
- Record every agent message
- Capture all tool executions
- Log LLM inputs and outputs
- Build a complete trace hierarchy
- Measure execution times


In [ ]:
mlflow.autogen.autolog()
print("✅ MLflow AutoGen autologging enabled")
print("📊 All agent interactions will now be automatically traced")

---

# Section 2: Define Tools & Agents

---

## 📌 STEP 6: Define Tool Functions

**What this does:**
- Creates mock tool functions that agents can call
- Implements database operations (`list_tables`, `execute_sql`)
- Implements web search functionality (`web_search`)
- Provides a schema metadata function for the DB agent
- Defines system prompts for each specialized agent

**💡 Key Concept:**
In a multi-agent system, tools are the "hands" of the agents. Each tool is an async function that:
- Has a clear docstring (agents use this to understand what the tool does)
- Returns a string result
- Can be assigned to specific agents based on their role


**💡 Production Note:**
In a real application, replace these with:
- `list_tables()` → Actual database introspection (SQLAlchemy)
- `execute_sql()` → Parameterized queries with SQL injection protection
- `web_search()` → Real search APIs (Tavily, Bing, Google Custom Search)

In [19]:
async def list_tables() -> str:
    """Lists all available tables in the database."""
    return "Tables: [users, software_licenses, purchase_orders]"


async def execute_sql(query: str) -> str:
    """Executes a SQL query and returns the results."""
    if "software_licenses" in query:
        return "Query Result: We have 15 active Illustrator licenses and 10 Photoshop licenses."
    return "Query Result: No relevant data found."


async def web_search(query: str) -> str:
    """Simulates a web search for public information."""
    return "Web Result: Adobe Illustrator's key tools include the 'Pen Tool', 'Mesh Tool', and 'Shape Builder'. It is vector-based software."


print("✅ Tool functions defined successfully")
print("🛠️  Available tools: list_tables, execute_sql, web_search")

✅ Tool functions defined successfully
🛠️  Available tools: list_tables, execute_sql, web_search


---

## 📌 STEP 7: Create Worker Agent Factories

**What this does:**
- Defines factory functions to create three specialized worker agents
- Each factory is decorated with `@mlflow.trace()` for observability
- Assigns specific tools to each agent based on their role
- Configures unique system messages to define agent behavior

**🔍 What to Observe:**
- `@mlflow.trace(name="...")`: Each factory gets its own trace span
- Each agent receives the same `model_client` but different `tools` and `system_message`
- Agent names (`web_surfer`, `db_specialist`, `report_writer`) are used in traces

**✅ Expected Output:**
```
✅ All Agent Factories and Atomic Tools Defined and Traced
```

In [17]:
@mlflow.trace(name="Web_Agent_Factory")
async def initialize_web_agent():
    """Traced function to create the Web Surfer Agent."""
    agent = AssistantAgent(
        name="web_surfer",
        model_client=model_client,
        tools=[web_search],
        system_message="You are a Web Researcher. Use the web_search tool for external facts and definitions.",
    )
    return agent

@mlflow.trace(name="Summary_Agent_Factory")
async def initialize_summary_agent():
    """Traced function to create the Report Writer Agent."""
    agent = AssistantAgent(
        name="report_writer",
        model_client=model_client,
        system_message="You are a Lead Technical Writer. Take raw findings and create a clean, professional executive summary in Markdown.",
    )
    return agent

@mlflow.trace(name="DB_Agent_Factory")
async def initialize_db_agent():
    """Traced function to create the Database Specialist Agent."""
    agent = AssistantAgent(
        name="db_specialist",
        model_client=model_client,
        tools=[list_tables, execute_sql],
        system_message="You are a Database Expert. Query internal tables. Use SQL tools provided."
    )
    return agent

print("✅ All Agent Factories and Atomic Tools Defined and Traced")

✅ All Agent Factories and Atomic Tools Defined and Traced


---

## 📌 STEP 8: Build the Master Orchestrator

**What this does:**
- Creates the master orchestrator agent that coordinates all workers
- Wraps worker agents as tools using `AgentTool`
- Configures the orchestrator's decision-making logic
- Sets up hierarchical tracing for the entire workflow

**🔍 What to Observe:**
- `@mlflow.trace(name="Manager_Agent")`: This creates the root trace span
- Worker agents are wrapped with `AgentTool(...)`
- `return_value_as_last_message=True`: Workers return their final answer
- The orchestrator gets access to three tools: `ask_db_specialist`, `ask_web_surfer`, `ask_report_writer`


**🌳 Trace Hierarchy:**
```
Orchestrator_Workflow_Run (root)
├── Manager_Agent
│   ├── Web_Agent_Factory
│   ├── DB_Agent_Factory
│   └── Summary_Agent_Factory
└── [Tool executions will appear here]
```

In [18]:
# --- INITIALIZE ALL AGENTS AND WRAP THEM AS TOOLS ---
@mlflow.trace(name="Manager_Agent")
async def initialize_orchestrator():
    # Call the traced factory functions
    db_agent = await initialize_db_agent()
    web_agent = await initialize_web_agent()
    summary_agent = await initialize_summary_agent()
    
    # Wrap agents as tools that the orchestrator can call
    db_tool = AgentTool(
        agent=db_agent,
        return_value_as_last_message=True
    )
    web_tool = AgentTool(
        agent=web_agent,
        return_value_as_last_message=True
    )
    summary_tool = AgentTool(
        agent=summary_agent,
        return_value_as_last_message=True
    )
    
    # --- CREATE MASTER ORCHESTRATOR ---
    orchestrator_prompt = """
    You are the **Master Orchestrator**. You must delegate the work to your tools.
    Your process must be:
    1. Determine what the user needs.
    2. Delegate research tasks to `ask_web_surfer` and `ask_db_specialist`.
    3. Use `ask_report_writer` to synthesize the final answer.
    """
    orchestrator = AssistantAgent(
        name="orchestrator",
        model_client=model_client,
        system_message=orchestrator_prompt,
        tools=[web_tool, db_tool, summary_tool],
        max_tool_iterations=10,
    )
    return orchestrator

print("✅ Orchestrator factory function defined")
print("🎯 Orchestrator will coordinate: web_surfer, db_specialist, report_writer")

✅ Orchestrator factory function defined
🎯 Orchestrator will coordinate: web_surfer, db_specialist, report_writer


---

# Section 3: Execute the Multi-Agent Workflow

---

## 📌 STEP 9: Run the Orchestration Workflow

**What this does:**
- Executes the complete multi-agent system with a sample query
- Displays the agent conversation in real-time using RichConsole
- Automatically captures the entire execution in MLflow traces
- Demonstrates the orchestrator delegating to specialized agents

**💡 Key Concept:**
The `@mlflow.trace()` decorator on `run_orchestration_workflow` creates a root span that contains all sub-agent activities. This hierarchical tracing allows you to:
- See the complete execution flow
- Identify bottlenecks
- Debug failures at any level
- Analyze agent decision-making

**📊 Expected Execution Flow:**

```
User Query: "What are Illustrator's key tools and our licenses?"
    ↓
Orchestrator: Analyzes query → Needs web info + DB data
    ↓
├─→ Web Surfer: "Find Illustrator key tools"
│   └─→ Returns: "Pen Tool, Mesh Tool, Shape Builder..."
│
├─→ DB Specialist: "Check license inventory"
│   └─→ Returns: "15 Illustrator licenses, 10 Photoshop..."
│
└─→ Report Writer: "Synthesize findings"
    └─→ Returns: Professional markdown report
```

**⚠️ Note:**
This cell will take 10-30 seconds to complete as it involves multiple LLM calls.

In [20]:
# ============================================================================
# MAIN EXECUTION
# ============================================================================

# Run the workflow
@mlflow.trace(name="Orchestrator_Workflow_Run")
async def run_orchestration_workflow(user_query: str):
    """Traces the entire workflow execution under a single root run."""
    orchestrator = await initialize_orchestrator()
    print(f"\n🚀 Starting Orchestration Workflow for: {user_query}")
    
    # Run the stream to display output and concurrently trace all actions
    await RichConsole(orchestrator.run_stream(task=user_query))
    
    print("\n🏁 Workflow Complete. Check MLflow UI for the full trace hierarchy!")

# --- RUN IT ---
task_query = "What are the key tools in Illustrator, and do we have licenses for them in our database?"

await run_orchestration_workflow(task_query)


🚀 Starting Orchestration Workflow for: What are the key tools in Illustrator, and do we have licenses for them in our database?


╭────────────────────────────────────────────────────────────────────────────────────────────────────────── user ─╮
│ What are the key tools in Illustrator, and do we have licenses for them in our database?                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/Users/aman/Desktop/mlflow-research/venv/lib/python3.11/site-packages/autogen_agentchat/agents/_assistant_agent.py:1109: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  model_result = await model_client.create(


╭─ orchestrator ──────────────────────────────────────────────────────────────────────────────────────────────────╮
│ [FunctionCall(id='call_UJaNS1V1PdPAqfK8REojlig4', arguments='{"task": "Find the key tools available in Adobe    │
│ Illustrator."}', name='web_surfer'), FunctionCall(id='call_lyJSyXYtUr4M7F04VaFbic8U', arguments='{"task":       │
│ "Check if we have licenses for Adobe Illustrator in our database."}', name='db_specialist')]                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────────────────────────────────────────────────────────────── user ─╮
│ Find the key tools available in Adobe Illustrator.                                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─ report_writer ─────────────────────────────────────────────────────────────────────────────────────────────────╮
│ # Executive Summary                                                                                             │
│                                                                                                                 │
│ This document highlights key tools in Adobe Illustrator and provides an overview of database structure relevant │
│ to software management.                                                                                         │
│                                                                                                                 │
│ ## Adobe Illustrator Key Tools                                                                                  │
│ Adobe Illustrator, a leading vector graphics software, offers a range of powerful tools to enhance design       │
│ workflows. Notable tools include:                                                                               │
│                                                                                                                 │
│ - **Pen Tool**: Enables precise creation of paths and curves for detailed vector illustrations.                 │
│ - **Mesh Tool**: Facilitates the creation of complex gradients and realistic shading by manipulating mesh       │
│ points.                                                                                                         │
│ - **Shape Builder Tool**: Simplifies the process of combining, merging, or subtracting shapes to create custom  │
│ designs efficiently.                                                                                            │
│                                                                                                                 │
│ These tools empower designers to produce high-quality, scalable graphics for various creative projects.         │
│                                                                                                                 │
│ ## Database Structure Overview                                                                                  │
│ The database supporting software management is structured with key tables that ensure efficient organization    │
│ and tracking of resources. Key tables include:                                                                  │
│                                                                                                                 │
│ - **users**: Stores information about system users, such as names, roles, and access levels.                    │
│ - **software_licenses**: Tracks details of software licenses, including license types, expiration dates, and    │
│ associated users.                                                                                               │
│ - **purchase_orders**: Maintains records of software-related purchases, including order details, vendors, and   │
│ costs.                                                                                                          │
│                                                                                                                 │
│ This database structure provides a robust foundation for managing software assets and user access effectively.  │
│                                                                                                                 │
│ By combining advanced design tools in Adobe Illustrator with a well-structured database, organizations can      │
│ streamline creative workflows and software management processes.                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ orchestrator ──────────────────────────────────────────────────────────────────────────────────────────────────╮
│ [FunctionExecutionResult(content='# Executive Summary\n\nThis document highlights key tools in Adobe            │
│ Illustrator and provides an overview of database structure relevant to software management.\n\n## Adobe         │
│ Illustrator Key Tools\nAdobe Illustrator, a leading vector graphics software, offers a range of powerful tools  │
│ to enhance design workflows. Notable tools include:\n\n- **Pen Tool**: Enables precise creation of paths and    │
│ curves for detailed vector illustrations.\n- **Mesh Tool**: Facilitates the creation of complex gradients and   │
│ realistic shading by manipulating mesh points.\n- **Shape Builder Tool**: Simplifies the process of combining,  │
│ merging, or subtracting shapes to create custom designs efficiently.\n\nThese tools empower designers to        │
│ produce high-quality, scalable graphics for various creative projects.\n\n## Database Structure Overview\nThe   │
│ database supporting software management is structured with key tables that ensure efficient organization and    │
│ tracking of resources. Key tables include:\n\n- **users**: Stores information about system users, such as       │
│ names, roles, and access levels.\n- **software_licenses**: Tracks details of software licenses, including       │
│ license types, expiration dates, and associated users.\n- **purchase_orders**: Maintains records of             │
│ software-related purchases, including order details, vendors, and costs.\n\nThis database structure provides a  │
│ robust foundation for managing software assets and user access effectively.\n\nBy combining advanced design     │
│ tools in Adobe Illustrator with a well-structured database, organizations can streamline creative workflows and │
│ software management processes.', name='report_writer', call_id='call_Y3MVLy5qgptUADgHKpX4hkDf',                 │
│ is_error=False)]                                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ orchestrator ──────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Adobe Illustrator is equipped with powerful tools such as the **Pen Tool**, **Mesh Tool**, and **Shape Builder  │
│ Tool**, which are essential for creating precise vector graphics, complex gradients, and custom shapes.         │
│                                                                                                                 │
│ Regarding licenses, our database includes tables like **users**, **software_licenses**, and                     │
│ **purchase_orders**, which are structured to manage software assets effectively. If you need further details on │
│ the licenses, let me know!                                                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


🏁 Workflow Complete. Check MLflow UI for the full trace hierarchy!


Trace(trace_id=tr-e3fc6d6f257ed24649f025eae44ed0a7)

---

# Section 4: Agent-as-a-Judge Evaluation

---

## 📌 STEP 10: Configure Databricks Environment for Judges

**What this does:**
- Validates that Databricks credentials are available
- Sets up LiteLLM-specific environment variables (required for Agent-as-a-Judge)
- Configures the Databricks model for judge LLM calls

**💡 Key Concept:**
Agent-as-a-Judge uses LiteLLM under the hood, which requires Databricks credentials in a specific format:
- `DATABRICKS_API_KEY` - Your Databricks personal access token
- `DATABRICKS_API_BASE` - Your Databricks workspace URL

In [ ]:
# ============================================================================
# Evaluation Part
# ============================================================================

# LiteLLM requires DATABRICKS_API_* prefixed variables for Databricks provider
os.environ["DATABRICKS_API_KEY"] = DATABRICKS_TOKEN
os.environ["DATABRICKS_API_BASE"] = DATABRICKS_HOST

# Judge model - use the same Databricks model
JUDGE_MODEL_NAME = DATABRICKS_MODEL

print("✅ Databricks environment configured for Agent-as-a-Judge")
print(f"🤖 Judge Model: {JUDGE_MODEL_NAME}")

---

## 🔍 What is Agent-as-a-Judge?

**Agent-as-a-Judge** represents a paradigm shift in LLM evaluation:

- **Traditional LLM-as-a-Judge**: Evaluates inputs/outputs only
- **Agent-as-a-Judge**: Acts as an autonomous agent with MCP tools to investigate complete execution traces


---

## 📌 STEP 11: Create Specialized Judges

**What this does:**
- Creates four specialized Agent-as-a-Judge evaluators
- Each judge analyzes a different aspect of the workflow execution
- Judges use MCP tools to explore traces autonomously

**💡 Key Concept:**
The `{{ trace }}` template variable is **critical** - it tells MLflow to:
- Use Agent-as-a-Judge (not simple LLM-as-a-Judge)
- Provide MCP tools (`GetTraceInfo`, `ListSpans`, `GetSpan`, `SearchTraceRegex`)
- Allow the judge to investigate traces like a debugger

**🔍 What to Observe:**
- `model=f"databricks/{JUDGE_MODEL_NAME}"`: LiteLLM format for Databricks
- Each judge has unique `instructions` defining what to analyze
- Judges return different output formats (ratings, scores, categorizations)

In [ ]:
# Create multiple judges for different evaluation aspects
# IMPORTANT: Use "databricks/<model-name>" format for Databricks models
orchestration_judge = make_judge(
    name="orchestration_analyzer",
    instructions=(
        "Analyze the orchestration pattern in {{ trace }}.\n\n"
        "Check for:\n"
        "1. Tool Usage: Did the orchestrator properly delegate to db_specialist, web_surfer, and report_writer?\n"
        "2. Sequencing: Were the agents called in a logical order?\n"
        "3. Completeness: Did all necessary sub-agents execute?\n"
        "4. Redundancy: Were there any unnecessary duplicate tool calls?\n\n"
        "Rate the orchestration as: 'excellent', 'good', 'needs_improvement', or 'poor'\n"
        "Provide specific span names and execution details."
    ),
    model=f"databricks/{JUDGE_MODEL_NAME}",
)

performance_judge = make_judge(
    name="performance_analyzer",
    instructions=(
        "Analyze the {{ trace }} for performance issues.\n\n"
        "Check for:\n"
        "- Total execution time and identify slow spans\n"
        "- Operations taking longer than expected\n"
        "- Opportunities for parallelization\n"
        "- Efficient delegation patterns\n\n"
        "Rate as: 'optimal', 'acceptable', or 'needs_improvement'\n"
        "Include timing details for key spans."
    ),
    model=f"databricks/{JUDGE_MODEL_NAME}",
)

tool_usage_judge = make_judge(
    name="tool_usage_evaluator",
    instructions=(
        "Analyze tool usage patterns in {{ trace }}.\n\n"
        "Check for:\n"
        "1. Correct tool selection: Did agents use appropriate tools (execute_sql, web_search, etc.)?\n"
        "2. Tool effectiveness: Did tools return useful results?\n"
        "3. Missing tools: Should additional tools have been used?\n"
        "4. Tool sequencing: Was the order of tool calls logical?\n\n"
        "Rate tool usage as: 'optimal', 'good', 'suboptimal', or 'poor'\n"
        "Provide specific examples from the trace."
    ),
    model=f"databricks/{JUDGE_MODEL_NAME}",
)

output_quality_judge = make_judge(
    name="output_quality_checker",
    instructions=(
        "Evaluate the final output quality in {{ trace }}.\n\n"
        "Analysis criteria:\n"
        "1. Completeness: Does the output address all parts of the user query?\n"
        "2. Accuracy: Are the facts correct based on the tool results?\n"
        "3. Synthesis: Did the report_writer properly combine DB and web results?\n"
        "4. Format: Is the output well-structured and professional?\n\n"
        "Score 1-100 for output quality and provide detailed feedback."
    ),
    model=f"databricks/{JUDGE_MODEL_NAME}",
)

print("✅ Four specialized judges created successfully")
print("🏛️  Judges: Orchestration, Performance, Tool Usage, Output Quality")
print(f"🤖 Using model: databricks/{JUDGE_MODEL_NAME}")

---

**💡 Quick Note on Judge Models:**

Agent-as-a-Judge uses **LiteLLM** for model calls, which supports:
- ✅ Databricks (what we use here)
- ✅ Azure OpenAI
- ✅ OpenAI
- ✅ Anthropic Claude
- ✅ Google Gemini
- ✅ AWS Bedrock
- ✅ others

**Model format for judges:**
```python
# Databricks
model="databricks/databricks-gpt-5-2"

# Azure OpenAI
model="azure:/gpt-4o"

# OpenAI  
model="gpt-4o"

# Anthropic
model="anthropic/claude-3-5-sonnet-20241022"

# See: https://docs.litellm.ai/docs/providers
```

📖 **Check compatibility:** [LiteLLM Providers](https://docs.litellm.ai/docs/providers) | [MLflow Judge Docs](https://mlflow.org/docs/latest/genai/eval-monitor/scorers/llm-judge/index.html)

---

---

## 📌 STEP 12: Define Evaluation Functions

**What this does:**
- Creates helper functions to run judges against traces
- Provides single-trace and batch evaluation capabilities
- Structures judge results for easy analysis

**🔍 What to Observe:**
- `evaluate_workflow_with_judges(trace_id)`: Evaluates a single trace with all 4 judges
- `evaluate_batch_traces(filter_string)`: Evaluates multiple traces (useful for A/B testing)
- Each judge returns a `feedback` object with `.value` and `.rationale`

**🎯 Feedback Object Structure:**
```python
feedback.value          # Rating (e.g., "excellent", "optimal", 95)
feedback.rationale      # Detailed explanation
feedback.trace_id       # Which trace was evaluated
feedback.source         # Judge metadata
feedback.create_time_ms # When evaluation occurred


In [29]:
async def evaluate_workflow_with_judges(trace_id: str):
    """
    Evaluate a single trace using multiple Agent-as-a-Judge scorers.
    
    Args:
        trace_id: The MLflow trace ID to evaluate
    """
    # Retrieve the trace
    trace = mlflow.get_trace(trace_id)
    
    # Run each judge
    judges = [
        ("Orchestration", orchestration_judge),
        ("Performance", performance_judge),
        ("Tool Usage", tool_usage_judge),
        ("Output Quality", output_quality_judge),
    ]
    
    results = {}
    
    for judge_name, judge in judges:
        feedback = judge(trace=trace)
        results[judge_name] = feedback
    
    return results


async def evaluate_batch_traces(filter_string: str = None):
    """
    Evaluate multiple traces using Agent-as-a-Judge.
    
    Args:
        filter_string: Optional filter for trace search (e.g., "timestamp > 1727174400000")
    """
    # Retrieve traces from MLflow
    if filter_string:
        traces = mlflow.search_traces(filter_string=filter_string)
    else:
        # Get recent traces from current experiment
        traces = mlflow.search_traces(max_results=5, order_by=["timestamp DESC"])
    
    all_results = []
    
    for trace in traces:
        # Evaluate with orchestration judge as example
        feedback = orchestration_judge(trace=trace)
        
        result = {
            "trace_id": trace.info.trace_id,
            "rating": feedback.value,
            "rationale": feedback.rationale,
        }
        all_results.append(result)
    
    return all_results

print("✅ Evaluation functions defined")
print("📊 Ready to evaluate single traces or batches")

✅ Evaluation functions defined
📊 Ready to evaluate single traces or batches


---

## 📌 STEP 13: Create Results Display Function

**What this does:**
- Creates a beautifully formatted display function for evaluation results
- Shows judge ratings, detailed analyses, and summaries
- Makes evaluation feedback human-readable and actionable



In [30]:
def print_evaluation_results(results: dict):
    """
    Print evaluation results in a human-readable format.
    
    Args:
        results: Dictionary of evaluation results from evaluate_workflow_with_judges
    """
    print("\n" + "="*100)
    print("🎯 AGENT-AS-A-JUDGE EVALUATION RESULTS")
    print("="*100)
    
    for idx, (judge_name, feedback) in enumerate(results.items(), 1):
        print(f"\n{'─'*100}")
        print(f"📊 Judge #{idx}: {judge_name}")
        print(f"{'─'*100}")
        
        print(f"\n🏆 Rating: {feedback.value.upper()}")
        print(f"📝 Trace ID: {feedback.trace_id}")
        print(f"🤖 Model: {feedback.source.source_id}")
        print(f"⏰ Evaluated at: {feedback.create_time_ms}")
        
        print(f"\n💭 Detailed Analysis:")
        print("─" * 100)
        
        # Word wrap the rationale for better readability
        rationale = feedback.rationale
        line_length = 95
        words = rationale.split()
        current_line = ""
        
        for word in words:
            if len(current_line) + len(word) + 1 <= line_length:
                current_line += word + " "
            else:
                print(current_line)
                current_line = word + " "
        
        if current_line:
            print(current_line)
    
    print("\n" + "="*100)
    print("✅ EVALUATION COMPLETE")
    print("="*100)
    
    # Summary
    print("\n📈 SUMMARY:")
    for judge_name, feedback in results.items():
        print(f"  • {judge_name:20s} → {feedback.value}")
    print()

print("✅ Results display function defined")
print("🎨 Ready to format and print evaluation feedback")

✅ Results display function defined
🎨 Ready to format and print evaluation feedback


## 📌 STEP 14: Execute Agent-as-a-Judge Evaluation

**What this does:**
- Retrieves the trace ID from the workflow execution (Step 9)
- Runs all four judges against the trace
- Displays beautifully formatted evaluation results

**💡 Key Concept:**
`mlflow.get_last_active_trace_id()` retrieves the most recent trace from the current notebook session. This is the trace created by our workflow execution in Step 9.

**⚠️ Note:**
This cell will take 30-60 seconds as each judge makes LLM calls to analyze the trace.

In [31]:
# Get the trace ID from the last run
trace_id = mlflow.get_last_active_trace_id()
print(f"📊 Evaluating trace: {trace_id}")
print("⏳ Running Agent-as-a-Judge evaluation (this may take 30-60 seconds)...\n")

# Evaluate the single trace with all judges
evaluation_results = await evaluate_workflow_with_judges(trace_id)

# Display the results in a beautiful format
print_evaluation_results(evaluation_results)

# Batch evaluation example (uncomment to use)
# batch_results = await evaluate_batch_traces()
# for result in batch_results:
#     print(f"Trace {result['trace_id']}: {result['rating']}")

📊 Evaluating trace: tr-e3fc6d6f257ed24649f025eae44ed0a7
⏳ Running Agent-as-a-Judge evaluation (this may take 30-60 seconds)...


🎯 AGENT-AS-A-JUDGE EVALUATION RESULTS

────────────────────────────────────────────────────────────────────────────────────────────────────
📊 Judge #1: Orchestration
────────────────────────────────────────────────────────────────────────────────────────────────────

🏆 Rating: NEEDS_IMPROVEMENT
📝 Trace ID: tr-e3fc6d6f257ed24649f025eae44ed0a7
🤖 Model: azure:/gpt-4o
⏰ Evaluated at: 1764673120266

💭 Detailed Analysis:
────────────────────────────────────────────────────────────────────────────────────────────────────
The trace illustrates the usage of multiple sub-agents: DB_Agent_Factory, Web_Agent_Factory, 
and Summary_Agent_Factory. Each sub-agent executes successfully but lacks integration 
verification in the orchestration. The sequence appears logical as web and database queries 
precede the summary generation. However, without an output from the orchestra

---

# 🎓 Conclusion & Next Steps

---

## What We Accomplished

Congratulations! You've successfully built, traced, and evaluated a sophisticated multi-agent AI system. Here's what you learned:

### ✅ Key Accomplishments

1. **Multi-Agent Architecture**
   - Built an orchestrator pattern with specialized worker agents
   - Implemented agent-as-tool delegation for hierarchical workflows
   - Created role-specific agents (DB Specialist, Web Surfer, Report Writer)

2. **MLflow Observability**
   - Enabled automatic tracing with `mlflow.autogen.autolog()`
   - Captured hierarchical execution flows across all agents
   - Created custom trace spans with `@mlflow.trace()` decorators

3. **Agent-as-a-Judge Evaluation**
   - Configured four specialized judges for comprehensive analysis
   - Leveraged MCP tools for autonomous trace exploration
   - Generated actionable feedback on orchestration, performance, tool usage, and quality

4. **Professional Development Practices**
   - Used factory patterns for clean agent initialization
   - Implemented async-first architecture for performance
   - Applied separation of concerns with specialized agents

---

## 🎯 What You Can Do Now

### 1. Explore MLflow UI

Navigate to `http://127.0.0.1:5000` and explore:

- **Experiments**: View all runs under "MLFlow-demo"
- **Traces Tab**: See the complete execution hierarchy
- **Timing Analysis**: Identify performance bottlenecks
- **Span Details**: Examine inputs/outputs for each operation
- **Evaluation Metrics**: View judge ratings and feedback

### 2. Customize Judges

Modify judge instructions to evaluate domain-specific criteria:

```python
custom_judge = make_judge(
    name="security_analyzer",
    instructions=(
        "Analyze {{ trace }} for security concerns.\n"
        "Check for: SQL injection risks, API key exposure, input validation..."
    ),
    model=f"databricks/{DATABRICKS_MODEL}"
)
```

### 3. Add Real Integrations

Replace mock functions with production tools:

**Database Integration:**
```python
import sqlalchemy

async def execute_sql(query: str) -> str:
    engine = sqlalchemy.create_engine(DATABASE_URL)
    with engine.connect() as conn:
        result = conn.execute(sqlalchemy.text(query))
        return str(result.fetchall())
```

**Web Search Integration:**
```python
from tavily import TavilyClient

async def web_search(query: str) -> str:
    client = TavilyClient(api_key=TAVILY_API_KEY)
    result = client.search(query)
    return result['answer']
```

### 4. Batch Evaluation

Compare multiple workflow executions:

```python
# Run the same query 5 times with different configurations
for config in configurations:
    await run_orchestration_workflow(query, config)

# Evaluate all runs
batch_results = await evaluate_batch_traces()

# Find the best configuration
best_run = max(batch_results, key=lambda x: x['rating'])
```

### 5. Production Deployment

Package this as an MLflow model:

```python
import mlflow.pyfunc

class MultiAgentModel(mlflow.pyfunc.PythonModel):
    async def predict(self, context, model_input):
        orchestrator = await initialize_orchestrator()
        result = await orchestrator.run(task=model_input['query'])
        return result

mlflow.pyfunc.log_model("multi_agent_orchestrator", python_model=MultiAgentModel())
```

---

## 📚 Additional Resources

### Documentation

- [MLflow Agent-as-a-Judge Documentation](https://mlflow.org/docs/latest/genai/eval-monitor/scorers/llm-judge/agentic-overview/)
- [AutoGen v0.4 Documentation](https://microsoft.github.io/autogen/)
- [MLflow Tracing Guide](https://mlflow.org/docs/latest/llms/tracing/index.html)
- [Databricks Foundation Model APIs](https://docs.databricks.com/en/machine-learning/foundation-models/index.html)

### Tutorials & Examples

- [AutoGen Multi-Agent Patterns](https://microsoft.github.io/autogen/docs/tutorial/introduction)
- [MLflow GenAI Examples](https://github.com/mlflow/mlflow/tree/master/examples/genai)
- [Building Production LLM Applications](https://www.databricks.com/blog/building-production-ready-llm-applications)

### Community

- [MLflow GitHub](https://github.com/mlflow/mlflow)
- [AutoGen GitHub](https://github.com/microsoft/autogen)
- [MLflow Community Slack](https://mlflow.org/slack)

---

## 🐛 Troubleshooting Guide

### Common Issues

**Issue: "Resource not found" or 401 error when running agents**
- **Solution**: Verify your Databricks credentials:
  - Check `DATABRICKS_HOST` is correct (include `https://`)
  - Check `DATABRICKS_TOKEN` is a valid personal access token
  - Ensure the model endpoint is available in your workspace

**Issue: Jupyter crashes with too many logs**
- **Solution**: Add logging suppressors at the top of the notebook:
  ```python
  import logging
  logging.getLogger("litellm").setLevel(logging.WARNING)
  ```

**Issue: Judges return empty/None ratings**
- **Solution**: 
  - Check Databricks credentials: `DATABRICKS_API_BASE` and `DATABRICKS_API_KEY`
  - Ensure the model is available: `databricks-meta-llama-3-3-70b-instruct`
  - Verify the model endpoint is responding

**Issue: MLflow UI connection refused**
- **Solution**: Start the MLflow server:
  ```bash
  uv run mlflow server --host 127.0.0.1 --port 5000
  ```

**Issue: Traces not appearing in UI**
- **Solution**: 
  - Verify tracking URI: `mlflow.get_tracking_uri()`
  - Check experiment exists: `mlflow.search_experiments()`
  - Ensure `mlflow.autogen.autolog()` was called before agent creation

**Issue: Model not found error**
- **Solution**: 
  - List available models: Check Databricks workspace for available Foundation Models
  - Try alternative models: `databricks-mixtral-8x7b-instruct` or `databricks-dbrx-instruct`

---

## 💡 Best Practices

### 1. Trace Organization
- Use descriptive trace names: `@mlflow.trace(name="descriptive_name")`
- Group related operations under parent spans
- Add metadata to traces for filtering: `mlflow.set_tag("version", "1.0")`

### 2. Judge Design
- Write specific, measurable evaluation criteria
- Use consistent rating scales across judges
- Include examples in judge instructions for clarity

### 3. Agent Prompting
- Give agents clear roles and responsibilities
- Specify expected output formats
- Include few-shot examples for complex tasks

### 4. Performance Optimization
- Use async/await for parallel agent execution
- Cache expensive operations (schema loading, embeddings)
- Implement timeout limits on tool calls

---

## 🙏 Feedback & Contributions

Have questions or suggestions? 

- **Issues**: Open an issue on [MLflow GitHub](https://github.com/mlflow/mlflow/issues)
- **Discussions**: Join the [MLflow Community Slack](https://mlflow.org/slack)
- **Contributions**: PRs welcome on [AutoGen](https://github.com/microsoft/autogen) and [MLflow](https://github.com/mlflow/mlflow)

---

## 🎉 You Did It!

You've mastered:
- ✅ Multi-agent architectures with AutoGen v0.4
- ✅ MLflow tracing for complete observability
- ✅ Agent-as-a-Judge for automated evaluation
- ✅ Production-ready AI system design patterns

**Next Challenge**: Build your own multi-agent system for your use case! 🚀

---

**Happy Building! 🛠️**

---

## 📝 Notebook Information

**Title**: Multi-Agent Orchestration with MLflow Tracing and Agent-as-a-Judge Evaluation  
**Version**: 1.1  
**Created**: December 2024  
**Updated**: February 2025  
**Framework**: AutoGen v0.4 + MLflow 3.6+  
**Model**: Databricks Foundation Models (GPT-5.2)  

**Sections**:
1. Setup & Configuration (Steps 1-5)
2. Define Tools & Agents (Steps 6-8)
3. Execute Workflow (Step 9)
4. Agent-as-a-Judge Evaluation (Steps 10-13)
5. Run Evaluation & Analyze (Step 14)

**Total Steps**: 14  
**Estimated Completion Time**: 30-45 minutes  
**Difficulty**: Intermediate

---

*This tutorial is based on the MLflow Agent-as-a-Judge documentation and AutoGen v0.4 multi-agent patterns.*